In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[38]:


import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import sys
import time



#Input file must be given to run the code.
#For Example Run(in terminal) 'python3 Bioinformatics input.xlsx'

#Filename =sys.argv[1] # Read input file 
df = pd.read_excel('updatedBigDataInput.xlsx')# Read EXCEL Files and storing in database 'df'


data= pd.DataFrame()# Creating New Data Frame 


# Storing Usefull information From the database 'df' 
data['Entry']=df['Entry'] #Storing Entry
data['Entry name']=df['Entry name']  #Storing Entry Name
data['Protein names']=df['Protein names']  #Storing Protein Names
data['Gene names']=df['Gene names'] # Storing Gene Names
data['Length']=df['Length'] # Storing Length
data['Organism']=df['Organism'] # Storing Type of Organism
data['Mass']=df['Mass'] #Storing Mass
data['Status']=df['Status']

#Now removing the extra data we dont need and containg only organism 'Human'
human_data=data[data['Organism'].str.contains("Homo sapiens")] #Selecting only organism which contains 'Homo sapiens'
data= human_data

data['Disulfide bond']=df['Disulfide bond'] # Storing Disulfide bond 
data['Glycosylation']= df['Glycosylation']# Storing Glycosylation Position
data=data.dropna().reset_index(drop=True) # Drop all rows which contains Not a number and reset the index 

# Working on getting the relative positions of the Disulfide bond
disulfide_column1= data['Disulfide bond'] # Storing data temporary as 'disulfide_column'
disulfide_column2= data['Disulfide bond']

def get_sulfide_value(newdata):
    return re.findall('\d+\..\d+',newdata)

bond= disulfide_column1.apply(get_sulfide_value) # Function call which gives all the positons 
data['Disulfide bond']=disulfide_column2.apply(get_sulfide_value)# Storing extracted disulfide value in data Frame.

In [3]:
def get_int_value(data):
    return re.findall('\d+',data)
def get_float_value(data):
    return re.findall('\d+\.\d+',data)

In [4]:
Glyco_data= data['Glycosylation'] # Storing data temporary as 'Glyco_data'
def getGlycoNLinked(data):   #Making a Function to get the positions
    return re.findall('CARBOHYD \d+',data) # The return data will contain 'CARBOHYD position #'
data['Glycosylation']= Glyco_data.apply(getGlycoNLinked) # the return data will be like 'CARBOHYD 110'
#Now Removing the extra word 'N-linked' and getting all positions of Glycosylation.
temp_data= data['Glycosylation'].astype(str)
data['Glycosylation']= temp_data.apply(get_int_value) 


# Creating a temporary dataframe for total length in columns and length in rows(like number of pairs in each index).
a= pd.DataFrame()
b= pd.DataFrame()
a =pd.Series(data['Glycosylation'])
b =pd.Series(data['Disulfide bond'])
lenB=b.str.len()
lenA=a.str.len()

rel_pos = pd.Series(a.size,dtype=np.str)
for i in range(a.size):
    rel_pos[i]=''
    
InsidePairs=pd.Series(b.size,dtype=np.str)
InsidePairsLength=np.zeros(b.size,dtype=np.int)
for i in range(a.size):
    InsidePairs[i]=''

# Now using loop and properties of Python library 're' to get values and finding positions between disulphide bond and glycosation.
i=0
while i<b.size:
    for j in range(lenA[i]):
        Position=int(a[i][j])
        rel_pos[i]=rel_pos[i]+str(a[i][j])+'{|'
        for k in range(lenB[i]):
            split = re.findall('\d+',b[i][k]) 
            firstPair= int(split[0])
            secondPair=int(split[1])
            if Position< firstPair:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            elif Position>= firstPair and Position<=secondPair:
                rel_pos[i]=  rel_pos[i]+'i' + str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'i'+ str(Position-secondPair)
                InsidePairs[i]= InsidePairs[i] + str('('+ str(firstPair)+','+str(secondPair)+')')
                InsidePairsLength[i]=InsidePairsLength[i]+ (secondPair-firstPair)
            else:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            rel_pos[i]=rel_pos[i]+'|'    
        rel_pos[i]=rel_pos[i]+'}'
    i=i+1
data['rel_pos']= rel_pos
data['In_Gly_Dis_Pair']=InsidePairs

In [5]:
#Caluclating Score of each pairs of disulfide bonds
# O score if other pair outside.
# 0.5 score if other pair half inside and half outside.
# 1.0 score if other pair full inside.
array = pd.Series(a.size,dtype=np.str)
for x in range(a.size):
    array[x]=''

for i in range(0,len(data)):
    temp_array = np.zeros(lenB[i])
    for j in range(0,lenB[i]):
        for k in range(j+1,lenB[i]):
            if k!=lenB[i]:
                split1 = re.findall('\d+',data['Disulfide bond'][i][j]) 
                p1e1=int(split1[0])
                p1e2=int(split1[1])
                split2 = re.findall('\d+',data['Disulfide bond'][i][k]) 
                p2e1=int(split2[0])
                p2e2=int(split2[1])
            
                if(p2e1>p1e2):
                    temp_array[j]=temp_array[j]+0
                    temp_array[k]=temp_array[k]+0
                elif (p2e1<p1e2) and (p2e2>p1e2):
                    temp_array[j]=temp_array[j]+0.5
                    temp_array[k]=temp_array[k]+0.5    
                elif (p2e1<p1e2) and (p2e2<p1e2):
                    temp_array[j]=temp_array[j]+1.0
                    temp_array[k]=temp_array[k]+0.0
        array[i]=array[i]+str(temp_array[j])+','
data['Disulfide_score']= array.apply(get_float_value)

In [6]:
#Claculating disulfide pair score average 
disulfie_pair_score_average = np.zeros(len(data),dtype=np.float)
for i in range(len(data)):
    for j in range(len(data['Disulfide_score'][i])):
        disulfie_pair_score_average[i]= disulfie_pair_score_average[i]+ float(data['Disulfide_score'][i][j])
data['disulfie_pair_score_average']=disulfie_pair_score_average

In [7]:
## Calculating Interbond Distance
interbond_distance= pd.Series(a.size,dtype= np.str)

for i in range(a.size):
    interbond_distance[i]=''

for i in range(a.size):
    
    if lenB[i]==0:
        interbond_distance[i]='No pair'
        
    if lenB[i]==1:
        interbond_distance[i]='Only One pair'
    
    for r in range(lenB[i]-1):
        split1= re.findall('\d+',data['Disulfide bond'][i][r])
        split2= re.findall('\d+',data['Disulfide bond'][i][r+1])
        interbond_distance[i]= interbond_distance[i]+ str(int(split2[0])-int(split1[1]))+ '|'
        
data['interbond_distance']=interbond_distance

#Intrabond distance formation
intrabond=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    intrabond[i]=''
j=0;
while j<b.size:
    for k in range(lenB[j]):
        split = re.findall('\d+',b[j][k]) 
        firstPair= int(split[0])
        secondPair=int(split[1])
        intrabond[j]=intrabond[j]+str(secondPair-firstPair)+'|'    
    j=j+1
data['intrabond']=intrabond


#number of pairs of disuphide bonds
data['total sulphide bonds']=lenB

#number of glcosylation positions
data['total glyco positions']=lenA


#1st disulphide pair to N-terminus distance
first_pos = pd.Series(a.size,dtype=np.int)
for x in range(a.size):
    if(data['Disulfide bond'][x]!=[]):
        split = re.findall('\d+',data['Disulfide bond'][x][0]) 
        firstPair= int(split[0])
        first_pos[x]= firstPair
data['first_position_occurance']=first_pos


#last disulphide pair to C-terminus distance
last_pos= pd.Series(a.size,dtype=np.int)
for x in range(a.size):
    if(data['Disulfide bond'][x]!=[]):
        split= re.findall('\d+',data['Disulfide bond'][x][(lenB[x]-1)])
        lastPair= int(split[1])
        last_pos[x] = lastPair 
data['last_position_occurance']=last_pos

#Calculate average distance of intrabond distance
average= data['intrabond'].apply(get_int_value)
lenIntra=average.str.len()

averageintrabond= np.zeros(a.size)
for i in range(a.size):
    totalsum=0
    for j in range(lenIntra[i]):
        totalsum= totalsum+int(average[i][j])
    averageintrabond[i]=totalsum/lenIntra[i]
data['average_Intrabond']=averageintrabond


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in long_scalars


In [8]:
#calculating average distance of those pairs of disulphide bonds that have Glycosylation Inside.
temp_length=pd.Series(b.size,dtype=np.str)
#for i in range(a.size):
#    temp_length[i]=''
inside_length = np.zeros(a.size,dtype=np.int)
for r in range(a.size):
    inside_length[r]=len(re.findall('\d+',data['In_Gly_Dis_Pair'][r]))/2
    temp_length[r]=re.findall('\d+',data['In_Gly_Dis_Pair'][r])
    if inside_length[r]!=0:
        InsidePairsLength[r]=InsidePairsLength[r]/inside_length[r]
data['avg_Inside_Pairs_Length']=InsidePairsLength

In [9]:
#If the glyco is inside then putting the sulphide position 'nil' at that point.
i=0
while i<b.size:
    for j in range(lenA[i]):
        Position=int(a[i][j])
        for k in range(lenB[i]):
            if bond[i][k]!='nil':
                split = re.findall('\d+',b[i][k]) 
                firstPair= int(split[0])
                secondPair=int(split[1])
                if Position>= firstPair and Position<=secondPair:
                    bond[i][k]='nil'
    i=i+1

In [10]:
#Calculating length between sulfide bonds.
temp=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    temp[i]=''

for i in range(len(bond)):
    for j in range(lenB[i]):
        if bond[i][j]!='nil':
            split= re.findall('\d+',bond[i][j])
            temp[i]=str(temp[i] + str(int(split[1])-int(split[0]))+'|')
data['glyco_outside_bond']=bond

In [11]:
#get the integer values.
intrabond_outside= temp.apply(get_int_value)
data['intrabond_glyco_outside_bond']=intrabond_outside

In [12]:
#calculating avegrage lenth of sulphide bonds which do not have glyco inside.

new =pd.Series(intrabond_outside)
newlen=new.str.len()
average= np.zeros(a.size)
for i in range(a.size):
    totalsum=0
    for j in range(newlen[i]):
        totalsum= totalsum+int(intrabond_outside[i][j])
    if newlen[i]!=0:
        average[i]=totalsum/newlen[i]
data['average_Intrabond_outside_glyco_bond']=average

In [13]:
data.to_excel('output.xlsx')

In [14]:
data

,Entry,Entry name,Protein names,Gene names,Length,Organism,Mass,Status,Disulfide bond,Glycosylation,...,intrabond,total sulphide bonds,total glyco positions,first_position_occurance,last_position_occurance,average_Intrabond,avg_Inside_Pairs_Length,glyco_outside_bond,intrabond_glyco_outside_bond,average_Intrabond_outside_glyco_bond
0,P04439,HLAA_HUMAN,"HLA class I histocompatibility antigen, A alph...",HLA-A HLAA,365,Homo sapiens (Human),"40,841",reviewed,"[125..188, 227..283]",[110],...,63|56|,2,1,125.0,283.0,59.500000,0,"[125..188, 227..283]","[63, 56]",59.500000
1,P01911,DRB_HUMAN,"HLA class II histocompatibility antigen, DRB1 ...",HLA-DRB1,266,Homo sapiens (Human),"29,966",reviewed,"[44..108, 146..202]",[48],...,64|56|,2,1,44.0,202.0,60.000000,64,"[nil, 146..202]",[56],56.000000
2,P01889,HLAB_HUMAN,"HLA class I histocompatibility antigen, B alph...",HLA-B HLAB,362,Homo sapiens (Human),"40,460",reviewed,"[125..188, 227..283]",[110],...,63|56|,2,1,125.0,283.0,59.500000,0,"[125..188, 227..283]","[63, 56]",59.500000
3,P08246,ELNE_HUMAN,Neutrophil elastase (EC 3.4.21.37) (Bone marro...,ELANE ELA2,267,Homo sapiens (Human),"28,518",reviewed,"[55..71, 151..208, 181..187, 198..223]","[88, 124, 173]",...,16|57|6|25|,4,3,55.0,223.0,26.000000,57,"[55..71, nil, 181..187, 198..223]","[16, 6, 25]",15.666667
4,P10144,GRAB_HUMAN,Granzyme B (EC 3.4.21.79) (C11) (CTLA-1) (Cath...,GZMB CGL1 CSPB CTLA1 GRB,247,Homo sapiens (Human),"27,716",reviewed,"[49..65, 142..209, 173..188]","[71, 104]",...,16|67|15|,3,2,49.0,188.0,32.666667,0,"[49..65, 142..209, 173..188]","[16, 67, 15]",32.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,A8MPY1,GBRR3_HUMAN,Gamma-aminobutyric acid receptor subunit rho-3...,GABRR3,467,Homo sapiens (Human),"54,272",reviewed,[184..198],[220],...,14|,1,1,184.0,198.0,14.000000,0,[184..198],[14],14.000000
2764,P28799,GRN_HUMAN,Progranulin (PGRN) (Acrogranin) (Epithelin pre...,GRN,593,Homo sapiens (Human),"63,544",reviewed,"[126..139, 133..149, 284..296, 290..306, 297.....","[118, 236, 265, 368, 530]",...,13|16|12|16|17|14|13|13|12|16|13|12|,12,5,126.0,416.0,13.916667,12,"[126..139, 133..149, 284..296, 290..306, 297.....","[13, 16, 12, 16, 17, 14, 13, 13, 16, 13, 12]",14.090909
2765,O00222,GRM8_HUMAN,Metabotropic glutamate receptor 8 (mGluR8),GRM8 GPRC1H MGLUR8,908,Homo sapiens (Human),"101,741",reviewed,"[64..106, 246..534, 369..384, 424..431, 516..5...","[95, 298, 452, 480, 565]",...,42|288|15|7|19|18|12|13|,8,5,64.0,569.0,51.750000,183,"[nil, nil, 369..384, 424..431, 516..535, 520.....","[15, 7, 19, 18, 12]",14.200000
2766,Q8IXB1,DJC10_HUMAN,DnaJ homolog subfamily C member 10 (EC 1.8.4.-...,DNAJC10 ERDJ5 UNQ495/PRO1012,793,Homo sapiens (Human),"91,080",reviewed,"[158..161, 480..483, 588..591, 700..703]",[530],...,3|3|3|3|,4,1,158.0,703.0,3.000000,0,"[158..161, 480..483, 588..591, 700..703]","[3, 3, 3, 3]",3.000000
